# Client per fer Prediccions als Models

Aquest notebook fa peticions als serveis web dels 4 models entrenats.

In [1]:
import requests
import json
import pandas as pd

# URL base del servei
BASE_URL = "http://localhost:5000"

# Comprovar que el servei està actiu
try:
    response = requests.get(f"{BASE_URL}/health")
    print("✓ Servei actiu!")
    print(json.dumps(response.json(), indent=2))
except:
    print("✗ El servei no està actiu. Executa primer: python scripts/app.py")

✓ Servei actiu!
{
  "models_loaded": 4,
  "status": "healthy"
}


In [2]:
# Llistar models disponibles
response = requests.get(f"{BASE_URL}/models")
models = response.json()
print("Models disponibles:")
print(json.dumps(models, indent=2))

Models disponibles:
{
  "count": 4,
  "models": [
    "logistic_regression",
    "svm",
    "decision_tree",
    "knn"
  ]
}


## Definir exemples de pingüins per provar

In [3]:
# Exemple 1: Pingüí Adelie típic
penguin_1 = {
    "island": "torgersen",
    "bill_length_mm": 39.1,
    "bill_depth_mm": 18.7,
    "flipper_length_mm": 181.0,
    "body_mass_g": 3750.0,
    "sex": "male"
}

# Exemple 2: Pingüí Gentoo típic
penguin_2 = {
    "island": "biscoe",
    "bill_length_mm": 50.0,
    "bill_depth_mm": 16.3,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5700.0,
    "sex": "male"
}

# Exemple 3: Pingüí Chinstrap típic
penguin_3 = {
    "island": "dream",
    "bill_length_mm": 46.5,
    "bill_depth_mm": 17.9,
    "flipper_length_mm": 192.0,
    "body_mass_g": 3500.0,
    "sex": "female"
}

# Exemple 4: Pingüí amb característiques intermitges
penguin_4 = {
    "island": "biscoe",
    "bill_length_mm": 45.2,
    "bill_depth_mm": 16.6,
    "flipper_length_mm": 191.0,
    "body_mass_g": 4250.0,
    "sex": "male"
}

penguins_test = [
    ("Pingüí 1 (esperat: Adelie)", penguin_1),
    ("Pingüí 2 (esperat: Gentoo)", penguin_2),
    ("Pingüí 3 (esperat: Chinstrap)", penguin_3),
    ("Pingüí 4 (cas intermedi)", penguin_4)
]

## Funció per fer prediccions

In [4]:
def predict_penguin(model_name, penguin_data, description=""):
    """
    Fa una predicció i mostra els resultats
    """
    url = f"{BASE_URL}/predict/{model_name}"
    
    try:
        response = requests.post(url, json=penguin_data)
        
        if response.status_code == 200:
            result = response.json()
            
            print(f"\n{'='*70}")
            if description:
                print(f"TEST: {description}")
            print(f"MODEL: {model_name.upper()}")
            print(f"{'='*70}")
            
            print(f"\nDades d'entrada:")
            for key, value in penguin_data.items():
                print(f"  - {key}: {value}")
            
            pred = result['prediction']
            print(f"\nPREDICCIÓ: {pred['species'].upper()}")
            print(f"Confiança: {pred['confidence']*100:.2f}%")
            
            print(f"\nProbabilitats per espècie:")
            for species, prob in sorted(pred['probabilities'].items(), key=lambda x: x[1], reverse=True):
                bar = "█" * int(prob * 40)
                print(f"  {species:12s}: {prob*100:6.2f}% {bar}")
            
            return result
        else:
            print(f"\n❌ Error {response.status_code}:")
            print(json.dumps(response.json(), indent=2))
            return None
            
    except Exception as e:
        print(f"\n❌ Error de connexió: {str(e)}")
        return None

## Provar tots els models amb els exemples

### 1. Regressió Logística

In [5]:
# Dues prediccions amb Regressió Logística
for desc, penguin in penguins_test[:2]:
    predict_penguin('logistic_regression', penguin, desc)


TEST: Pingüí 1 (esperat: Adelie)
MODEL: LOGISTIC_REGRESSION

Dades d'entrada:
  - island: torgersen
  - bill_length_mm: 39.1
  - bill_depth_mm: 18.7
  - flipper_length_mm: 181.0
  - body_mass_g: 3750.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 99.99%

Probabilitats per espècie:
  Adelie      :  99.99% ███████████████████████████████████████
  Chinstrap   :   0.01% 
  Gentoo      :   0.00% 

TEST: Pingüí 2 (esperat: Gentoo)
MODEL: LOGISTIC_REGRESSION

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 50.0
  - bill_depth_mm: 16.3
  - flipper_length_mm: 230.0
  - body_mass_g: 5700.0
  - sex: male

PREDICCIÓ: GENTOO
Confiança: 99.89%

Probabilitats per espècie:
  Gentoo      :  99.89% ███████████████████████████████████████
  Chinstrap   :   0.07% 
  Adelie      :   0.04% 


### 2. SVM

In [6]:
# Dues prediccions amb SVM
for desc, penguin in penguins_test[1:3]:
    predict_penguin('svm', penguin, desc)


TEST: Pingüí 2 (esperat: Gentoo)
MODEL: SVM

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 50.0
  - bill_depth_mm: 16.3
  - flipper_length_mm: 230.0
  - body_mass_g: 5700.0
  - sex: male

PREDICCIÓ: GENTOO
Confiança: 98.76%

Probabilitats per espècie:
  Gentoo      :  98.76% ███████████████████████████████████████
  Adelie      :   0.72% 
  Chinstrap   :   0.51% 

TEST: Pingüí 3 (esperat: Chinstrap)
MODEL: SVM

Dades d'entrada:
  - island: dream
  - bill_length_mm: 46.5
  - bill_depth_mm: 17.9
  - flipper_length_mm: 192.0
  - body_mass_g: 3500.0
  - sex: female

PREDICCIÓ: CHINSTRAP
Confiança: 99.08%

Probabilitats per espècie:
  Chinstrap   :  99.08% ███████████████████████████████████████
  Gentoo      :   0.55% 
  Adelie      :   0.37% 


### 3. Arbres de Decisió

In [7]:
# Dues prediccions amb Arbres de Decisió
for desc, penguin in [penguins_test[0], penguins_test[3]]:
    predict_penguin('decision_tree', penguin, desc)


TEST: Pingüí 1 (esperat: Adelie)
MODEL: DECISION_TREE

Dades d'entrada:
  - island: torgersen
  - bill_length_mm: 39.1
  - bill_depth_mm: 18.7
  - flipper_length_mm: 181.0
  - body_mass_g: 3750.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 100.00%

Probabilitats per espècie:
  Adelie      : 100.00% ████████████████████████████████████████
  Chinstrap   :   0.00% 
  Gentoo      :   0.00% 

TEST: Pingüí 4 (cas intermedi)
MODEL: DECISION_TREE

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 45.2
  - bill_depth_mm: 16.6
  - flipper_length_mm: 191.0
  - body_mass_g: 4250.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 100.00%

Probabilitats per espècie:
  Adelie      : 100.00% ████████████████████████████████████████
  Chinstrap   :   0.00% 
  Gentoo      :   0.00% 


### 4. KNN

In [8]:
# Dues prediccions amb KNN
for desc, penguin in [penguins_test[2], penguins_test[3]]:
    predict_penguin('knn', penguin, desc)


TEST: Pingüí 3 (esperat: Chinstrap)
MODEL: KNN

Dades d'entrada:
  - island: dream
  - bill_length_mm: 46.5
  - bill_depth_mm: 17.9
  - flipper_length_mm: 192.0
  - body_mass_g: 3500.0
  - sex: female

PREDICCIÓ: CHINSTRAP
Confiança: 100.00%

Probabilitats per espècie:
  Chinstrap   : 100.00% ████████████████████████████████████████
  Adelie      :   0.00% 
  Gentoo      :   0.00% 

TEST: Pingüí 4 (cas intermedi)
MODEL: KNN

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 45.2
  - bill_depth_mm: 16.6
  - flipper_length_mm: 191.0
  - body_mass_g: 4250.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 100.00%

Probabilitats per espècie:
  Adelie      : 100.00% ████████████████████████████████████████
  Chinstrap   :   0.00% 
  Gentoo      :   0.00% 


## Comparació de tots els models amb el mateix pingüí

In [10]:
# Provar tots els models amb el mateix pingüí
test_penguin = penguin_4

print("\n" + "#"*70)
print("# COMPARACIÓ DE TOTS ELS MODELS AMB EL MATEIX PINGÜÍ")
print("#"*70)

results_comparison = []
for model_name in ['logistic_regression', 'svm', 'decision_tree', 'knn']:
    result = predict_penguin(model_name, test_penguin)
    if result:
        results_comparison.append({
            'Model': model_name,
            'Predicció': result['prediction']['species'],
            'Confiança': f"{result['prediction']['confidence']*100:.2f}%"
        })

# Mostrar resum
print("\n" + "="*70)
print("RESUM DE PREDICCIONS")
print("="*70)
df_comparison = pd.DataFrame(results_comparison)
print(df_comparison.to_string(index=False))


######################################################################
# COMPARACIÓ DE TOTS ELS MODELS AMB EL MATEIX PINGÜÍ
######################################################################

MODEL: LOGISTIC_REGRESSION

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 45.2
  - bill_depth_mm: 16.6
  - flipper_length_mm: 191.0
  - body_mass_g: 4250.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 68.39%

Probabilitats per espècie:
  Adelie      :  68.39% ███████████████████████████
  Gentoo      :  29.79% ███████████
  Chinstrap   :   1.82% 

MODEL: SVM

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 45.2
  - bill_depth_mm: 16.6
  - flipper_length_mm: 191.0
  - body_mass_g: 4250.0
  - sex: male

PREDICCIÓ: ADELIE
Confiança: 85.65%

Probabilitats per espècie:
  Adelie      :  85.65% ██████████████████████████████████
  Gentoo      :  11.56% ████
  Chinstrap   :   2.79% █

MODEL: DECISION_TREE

Dades d'entrada:
  - island: biscoe
  - bill_length_mm: 45.2
  - bill_depth_mm